---
# Projectr : 하루시작 지하철 혼잡도 분석

### Description : 
- <a><span style = "color: #FFBE98">
    <b>BERT Model을 활용한 오늘의 주요 키워드 및 뉴스 추천하기</b>
  </a>

### Author : Zen Den

### Date : 2024. 06. 13. (Thu) ~

### Detail :
  - Crawling 평균 소요시간 <br><br>
    - 언론사별 랭킹뉴스의 언론사명, Title, Link: 1분 50초 <br><br>
    - 뉴스 기사의 본문: 15분 30초 <br><br>

  > (1) 환경 설정 및 라이브러리 설치
  - 웹 크롤링을 위한 requests와 BeautifulSoup, Selenium <br>
  - 데이터 처리를 위한 pandas <br>
  - 텍스트 처리를 위한 nltk <br>
  - 딥러닝 모델인 BERT를 활용하기 위한 transformers와 torch <br>
  - 자연어 처리 위한 sklearn

  > (2) Web Crawling
  - 네이버 뉴스 페이지를 요청하고, BeautifulSoup으로 HTML 파싱 <br>
  - 각 언론사별로 상위 5개 뉴스 기사 제목 및 Link 추출

  > (3) Keyword 추출
  - 각 뉴스 기사 본문을 가져와서 빈도수 높은 단어 추출 <br>
  - 불용어 제거 및 형태소 분석을 통해 주요 Keyword 도출

  > (4) 뉴스 추천 시스템 구축
  - BERT Model을 활용하여 각 뉴스 기사 Embedding 생성 <br>
  - 코사인 유사도를 사용하여 주요 Keyword와 유사한 뉴스 기사 추천

### Update: 
- 2024.06.13. (Thu) K.Zen : <br><br>
  - <span style = "color: cyan">
      <b>Deep Learning(BERT Model)</b>
      을 활용한 뉴스 추천하기
    </span>

---
# Environment Setting and Import Library Package

## Basic

In [3]:
import pandas as pd, numpy as np

## File System

In [4]:
import os; from datetime import datetime

## Crawling

In [5]:
import requests # 인터넷에서 Data를 가져오기 위한 Library (웹페이지에 접속하고 HTML 코드를 가져오기 위해 사용)
from bs4 import BeautifulSoup # 웹 페이지 내용을 분석하기 위한 Library (가져온 HTML 코드에서 우리가 필요한 정보를 추출하기 위해 사용)

import time # 대기 시간을 추가하기 위한 Library (요청 사이에 랜덤한 시간을 기다리기 위해 사용)
import random # Random한 대기 시간을 만들기 위한 Library
from tqdm import tqdm # Crawling 진행 상황을 체크하기 위한 Module (진행 상황을 시각적으로 보여주기 위해 사용)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## Bag of Words (BoW)

In [20]:
import nltk # Natural Language Toolkit (자연어 처리를 위해 사용)
from konlpy.tag import Okt
from collections import Counter # 단어의 빈도를 계산하기 위해 사용

# 1. nltk Data Download
nltk.download('punkt')

# 2. 한국어 불용어 사전
# ****************************************************************
## 한국어 불용어 모음집 불러오기
stopword_list = pd.read_csv("Data/updated_stopword.txt", header = None)
# ****************************************************************
stopword_list[0] = stopword_list[0].apply(lambda x: x.strip())
stopwords = stopword_list[0].to_numpy()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gwangyeong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Deep Learning - BERT

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer
import torch

# Web Crawling

In [9]:
# 1. 뉴스 Crawling
def get_news_links_by_press (url) :
  """
    headers:
    - 나는 bot이 아니고 사람임을 증명하는 부분이다.
      - 사용하지 않을 시 언론사에서 웹크롤링을 막을 수 있으니 주의할 것!
  """
  headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
  }
  response = requests.get(url, headers = headers) # url(페이지)에 접속
  soup = BeautifulSoup(response.content, 'html.parser') # HTML 코드를 파싱(분석)하여 soup 객체에 저장
  
  press_data = {}
  press_sections = soup.select('.rankingnews_box')
  
  for press_section in tqdm(press_sections, desc = "언론사별 뉴스 Crawling") :
    press_name = press_section.select_one('.rankingnews_name').get_text(strip = True)
    news_links = set()  # 중복 제거를 위한 set 사용
    for item in press_section.select('li a') :
      title = item.get_text(strip = True)
      link = item['href']
      if title and link and "동영상" not in title :  # Title이 존재하고 "동영상"이 포함되지 않은 경우에만 추가
        news_links.add((title, link))
    press_data[press_name] = list(news_links)[:5]  # 다시 list로 변환 후 상위 5개만 저장
    
    # *********************************************
    # 각 언론사별 뉴스 Crawling 후 대기 시간 추가
    time.sleep(random.uniform(0.5, 2.0))
    # *********************************************
  
  return press_data

In [10]:
base_url = 'https://news.naver.com/main/ranking/popularDay.naver'
press_news_data = get_news_links_by_press(base_url)

# 1. 뉴스 DataFrame 생성
news_list = []
for press_name, news_data in press_news_data.items() :
    for title, link in news_data :
        news_list.append([press_name, title, link])
news_df = pd.DataFrame(news_list, columns = ['Press', 'Title', 'Link'])

news_df.head()

언론사별 뉴스 Crawling: 100%|██████████| 83/83 [01:46<00:00,  1.28s/it]


,Press,Title,Link
0,경향신문,아내에 성인방송 출연 ‘협박·감금’ 전직 군인 징역 7년 구형,https://n.news.naver.com/article/032/000330235...
1,경향신문,이성윤 “특활비로 술먹고 민원실에 대변 본 검사들...공수처 조사해야”,https://n.news.naver.com/article/032/000330235...
2,경향신문,"‘김건희 명품백’ 폭로한 기자 “내 돈으로 샀다, 이제 돌려달라”",https://n.news.naver.com/article/032/000330236...
3,경향신문,"팝스타 아리아나 그란데, 하이브와 재계약···팬덤 플랫폼 위버스도 입점",https://n.news.naver.com/article/032/000330232...
4,경향신문,"독자 일정 일절 비공개…김건희 여사의 ‘조용한 순방’, 왜?",https://n.news.naver.com/article/032/000330235...


# Keyword 추출

In [21]:
# Chrome Browser와 Chrome Driver Version 확인 및 WebDriver 객체 생성
chrome_options = webdriver.ChromeOptions()
# ******************************************************
chrome_options.add_argument('headless') # Run chrome browser in the background
chrome_options.add_argument('window-size = 1920x1080')  # Chrome Browser Window Size
# ******************************************************
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)

# 1. 뉴스 기사 본문 가져오기 함수 정의
def get_article_text (url) :
    driver.get(url)

    # # **************************************************************************
    # # 기본 Path 설정
    # today = datetime.now().strftime('%Y%m%d')   # 오늘 날짜 가져오기
    # base_path = f"Data/ScreenShot/By_Press/{today}"

    # ## By_Press 폴더 생성
    # os.makedirs(base_path, exist_ok = True)

    # # 동일한 By_Press/today Folder가 존재하는지 확인하고, 존재한다면 하위에 새로운 Folder 생성
    # new_path = base_path
    # while os.path.exists(new_path) :
    #     new_path = os.path.join(base_path, str(len(os.listdir(base_path)) + 1)) # index는 폴더 내 파일 개수 + 1로 설정
    
    # # 새로운 Folder 생성
    # os.makedirs(new_path, exist_ok = True)

    # # 기본 File Name 설정
    # base_filename = news_df[news_df['Link'] == url]['Title'].values[0]
    # extension = ".png"
    # name_index = 1  # File Name에 추가될 숫자
    # new_filename = base_filename + extension

    # # File Path 생성
    # screenshot_name = os.path.join(new_path, new_filename)

    # # 동일한 File Name이 존재하는지 확인하고, 존재한다면 새로운 File Name 생성
    # while os.path.exists(screenshot_name) :
    #     new_filename = f"{base_filename}_{name_index}{extension}"
    #     screenshot_name = os.path.join(new_path, new_filename)
    #     name_index += 1
    
    # # Browser 최대화
    # driver.maximize_window()
    
    # # 현재 화면 Capture하기
    # driver.save_screenshot(screenshot_name)
    # # **************************************************************************

    html = driver.page_source
    article_soup = BeautifulSoup(html, "html.parser")
    content = article_soup.select_one("#contents")
    if content :
        # 공백과 HTML Tag 제거
        text =''.join(content.text.split())
        # *******************************************
        # 요청 후 임의의 시간만큼 대기 (Page Loaded)
        time.sleep(random.uniform(0.5, 2.0))
        # *******************************************
        
        return text

# 2. 뉴스 기사 본문 수집
content_pbar = tqdm(news_df['Link'], desc = "뉴스 기사 본문 Crawling 진행 중", unit = "Link")
news_df['content'] = [get_article_text(url) for url in content_pbar]

# Browser 종료 (모든 Tab 종료)
driver.quit()

# 3. 키워드 추출 함수 정의
def get_keywords (text, num_keywords = 10) :
    okt = Okt()
    tokens = okt.nouns(text)
    tokens = [word for word in tokens if word not in stopwords and len(word) > 1]
    return [word for word, freq in Counter(tokens).most_common(num_keywords)]

# 4. 각 언론사별 키워드 추출
text_pbar = tqdm(news_df['content'], desc = "Text 처리 중", unit = "기사")
news_df['keywords'] = [get_keywords(text) for text in text_pbar]
news_df['keywords'].head()

print(news_df['content'].head().values)
print('🍋' * 80)
print(news_df['keywords'].head().values)

Text 처리 중: 100%|██████████| 408/408 [02:04<00:00,  3.27기사/s]

['성인방송출연을요구하며아내를협박·감금한혐의로전직군인이인천지법에들어가고있다.연합뉴스제공아내가숨지기전성인방송출연을요구하며협박·감금한혐의로구속기소된30대전직군인에게검찰이징역7년을구형했다.검찰은인천지법형사5단독홍준서판사심리로14일열린결심공판에서감금과협박혐의로구속기소된A씨(37)에게징역7년을구형했다고밝혔다.A씨의선고공판은다음달12일열린다.검찰은“A씨는아내를지속해서감금·협박하고,사회관계망서비스(SNS)에음란사진을게시했다”며“피해자가정신적고통을호소하며유서를쓰고자살까지이르게해죄질이좋지않다”고구형아유를밝혔다.A씨측변호인은“A씨는피해자에게해를끼칠마음이없었고대화를시도하기위해집으로부른것”이라며“A씨는공소사실대부분을인정하고,진지하게반성하는점을참작해달라”고재판부에요청했다.검찰의구형전피해자의유족들은재판부에엄벌을요청했다.피해자의아버지는“A씨는딸에게성인방송을강요했고거부하니‘아버지에게나체사진을보내겠다’며협박했다”며“A씨는성인방송수입금으로고급차와명품옷·운동화로자신을과시하고다녔다”고말했다.이어“하나밖에없는딸을죽음으로내몬피고인에게엄벌을내려달라”고덧붙였다.A씨는2021년부터지난해까지30대아내B씨를자택에감금하고,성관계영상촬영과성인방송출연을강요한혐의로구속기소됐다.또성인방송촬영을거부하는B씨에게“나체사진을장인어른에게보내겠다”며협박한혐의도받는다.B씨는지난해12월피해를호소하는내용의유서를남기고숨졌다.A씨는2011년여성나체사진등을98차례인터넷에올린혐의(정보통신망이용촉진및정보보호등에관한법률상음란물유포)도받았다.직업군인이던A씨는2021년온라인에서불법영상물을공유했다가강제전역한것으로알려졌다.박준철기자terryus@kyunghyang.com기자프로필박준철기자구독구독중구독자0응원수0경향신문인천담당박준철기자입니다.“정부,인천공항적자땐한푼도안줘놓고···흑자땐절반가까이꿀꺽”윤대통령장모최은순씨,성남땅27억과징금항소심도패소경향신문의구독많은기자를구독해보세요!닫기Copyrightⓒ경향신문.Allrightsreserved.무단전재및재배포금지.이기사는언론사에서경제,사회,생활섹션으로분류했습니다.기사섹션분류안내기사의섹션정보는해당언론사의분류를

In [19]:
# # 불용어 추가
# # new_stopwords = [
# #     "구독", "기사", "기자", "언론사", "추천", "보기", "섹션", "뉴스", "마이", "어스"
# # ]
# new_stopwords = list(news_df['Press'].unique())

# # 불용어 파일 경로
# stopword_file_path = 'Data/updated_stopword.txt'

# # 불용어 파일 읽기
# with open(stopword_file_path, 'r', encoding = 'utf-8') as file :
#   stopwords = file.read().splitlines()

# # 기존 불용어 목록에 새 불용어 추가
# for word in new_stopwords :
#   if word not in stopwords :  # 중복되지 않도록 체크
#     stopwords.append(word)

# # 새로운 불용어 파일 경로
# new_stopword_file_path = 'Data/updated_stopword.txt'

# # 불용어 파일 저장
# with open(new_stopword_file_path, 'w', encoding = 'utf-8') as file :
#   for stopword in stopwords :
#     file.write(stopword + '\n')

# print("불용어 목록이 성공적으로 업데이트되고 새로운 파일에 저장되었습니다.")

불용어 목록이 성공적으로 업데이트되고 새로운 파일에 저장되었습니다.


# 뉴스 추천 시스템 구축

In [22]:
# 1. BERT 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

"""
    with torch.no_grad():
    - 블록을 사용하여 모델의 출력을 계산할 때 그래디언트를 저장하지 않도록 했습니다.
        - 이는 메모리 사용량을 줄이고 성능을 향상시킵니다.
"""
# 2. 문장 임베딩 생성 함수 정의
def get_bert_embedding (text) :
    inputs = tokenizer(
        text,
        return_tensors = 'pt',
        truncation = True,
        padding = True,
        max_length = 512
    )
    with torch.no_grad() :
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim = 1).detach().numpy()

# 3. 각 뉴스 기사 임베딩 생성
embed_pbar = tqdm(news_df['content'], desc = "Embedding 처리 중", unit = "text")
news_df['embedding'] = [get_bert_embedding(text) for text in embed_pbar]


"""
    keyword_text 생성 시, sum(news_df['keywords'], [])을 사용하여 List의 List를 평탄화한 후 Keyword를 결합합니다.
"""
# 4. 주요 Keyword 임베딩 생성
keyword_text = ' '.join(sum(news_df['keywords'], []))
keyword_embedding = get_bert_embedding(keyword_text)


"""
    np.vstack(df['embedding'].values:
    - numpy 배열로 변환하는 부분입니다.
        - 이는 embeddings 변수를 numpy 배열로 변환하여 코사인 유사도를 계산할 수 있게 합니다.
"""
# 5. 코사인 유사도 계산 및 뉴스 추천
def recommend_news (df, keyword_embedding, top_n = 10) :
    embeddings = np.vstack(df['embedding'].values)
    similarities = cosine_similarity(embeddings, keyword_embedding.reshape(1, -1)).flatten()
    df['similarity'] = similarities
    return df.nlargest(top_n, 'similarity')

recommended_news = recommend_news(news_df, keyword_embedding)
recommended_news[['Press', 'Title', 'similarity']]

Embedding 처리 중: 100%|██████████| 408/408 [02:24<00:00,  2.82text/s]


,Press,Title,similarity
291,프레시안,"홍준표 ""실체적 진실이면 이재명은 감옥, 꿰맞춘 수사라면 검찰은 궤멸""",0.739615
136,디지털타임스,"[속보]이재명 ""대북송금은 있을 수 없는 희대의 조작 사건으로 밝혀질 것""",0.737046
147,조선비즈,"380만원 넘는 디올백, 원가는 8만원… ‘노동 착취’ 실태 드러나",0.735331
24,이데일리,"거지가 한 달에 375만원 버는 나라…정부 ""절대 돈 주지 마라"" 경고",0.734711
138,디지털타임스,"""장인이 만든 줄 알았는데""…380만원짜리 디올 가방 원가에 `발칵`",0.733185
78,연합뉴스,"""성인방송 강요받다 숨진 내 딸…"" 법정서 아버지 절규",0.733176
122,MBC,"대통령실, '투르크멘 국견 선물' 언급하며 ""김 여사 동물사랑 반향""",0.731834
180,시사IN,국민 60%가 ‘석유 가능성’ 안 믿는 이유 [김은지의 뉴스IN],0.731606
238,전자신문,태국 거지의 한 달 수입은?… “375만원”,0.730998
74,국민일보,"‘신림 칼부림’ 조선, 2심 선고 전 “감형 한 번 도와주세요”",0.730795
